In [54]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
%matplotlib inline

In [55]:
from IPython.display import SVG
from urllib.request import urlretrieve
import zipfile
import collections

Loading csv files

In [ ]:
#dataset_TIST2015_Checkins: User ID, Venue ID, UTC time, Time zone
#dataset_TIST2015_POIs: Venue ID, Latitude, Longitude, Venue category name, Country code 
#dataset_TIST2015_Cities: Cityname, Latitude, Longitude, Vanue Category, Country code, Country name, City type
#dataset_UbiComp2016_UserProfile_{CITY}: User ID, Gender, Twitter friend count, Twitter follower count

#Scenario 1: Checkins og POI locations

checkin_cols = ['user_id', 'poi_id', 'timestamp', 'timezone']
checkin = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_Checkins.csv', sep=',', names=checkin_cols, encoding='latin-1')

city_cols = ['city_name', 'citycenter_latitude', 'citycenter_longitude', 'country_code', 'country_name', 'city_type']
cities = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_Cities.csv', sep=',', names=city_cols, encoding='latin-1')

venue_cols = ['poi_id', 'latitude', 'longitude', 'category', 'country_code']
pois = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_POIs.csv', sep=',', names=venue_cols, encoding='latin-1')


#Scenario 2: Users og Checkins

# Load each data set (users, movies, and ratings).
users_cols = ['user_id', 'gender', 'sex']
ratings_cols = ['user_id', 'venue_id', 'latitude', 'longitude', 'category' 'unix_timestamp']

#Scenario3: Users, Checkins og POI Locations
checkin_cols = ['poi_id', 'gender', 'sex']
venue_cols = ['venue_id', 'user_id', 'latitude', 'longitude', 'category' 'unix_timestamp']
user_cols = ['user_id', 'gender']




In [ ]:
dim1 = cities[['city_name', 'citycenter_longitude', 'citycenter_latitude', 'city_type', 'country_code']].drop(labels=[0], axis=0)

In [ ]:
dim2 = checkin[['user_id', 'poi_id', 'timestamp']].drop(labels=[0], axis=0)

In [ ]:
dim3 = pois[['poi_id', 'latitude', 'longitude', 'category', 'country_code']].drop(labels=[0], axis=0)

In [ ]:
dataset = dim2.merge(dim3, on='poi_id')#.merge(dim1, on='country_code')

In [ ]:
# Merge Users and Checkins. 
# However, our data does not allow us to merge users and checkin, because we only have data for users that have checked in NYC and Tokyo
#movielens = ratings.merge(movies, on='movie_id').merge(users, on='user_id')

In [ ]:
#Split into train and test
train_, test = train_test_split(dataset, test_size=0.1)

In [ ]:
ROW_COUNT = train_.shape[0]

In [ ]:
EMBEDDING_SIZE = 10
NUM_USERS = dataset['user_id'].nunique()
NUM_POI = dataset['poi_id'].nunique()

In [ ]:
dataset.head()

In [ ]:
UNIQUE_MOVIE_IDS = dataset['movie_id'].unique()

# Create Subset of Dataset

## Load in the dataset from csv files

In [3]:
checkin_cols = ['user_id', 'poi_id', 'timestamp', 'timezone']
checkins = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_Checkins.csv', sep=',', names=checkin_cols, encoding='latin-1')

venue_cols = ['poi_id', 'latitude', 'longitude', 'category', 'country_code']
pois = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_POIs.csv', sep=',', names=venue_cols, encoding='latin-1')



C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Function that extracts a subset

In [4]:
# Create a subset of a dataset with size n users
def create_checkin_subset(checkins_table, n):
    subset = np.array([[0,0,0,0],[0,0,0,0]])
    checkins_100_users = checkins_table['user_id'].sample(n=n, random_state=1)
    for i in checkins_100_users:
        subset = np.append(subset, np.asarray(checkins_table[checkins_table['user_id'] == i].values), 0)
    return pd.DataFrame(subset[2::], columns=['user_id', 'poi_id', 'timestamp', 'timezone'])

### Calling the function

In [5]:
# Initialize the subset method
checkins_100_users = create_checkin_subset(checkins, 25)
checkin_data = checkins_100_users.merge(pois, on='poi_id')
df = checkin_data.set_index('user_id').poi_id.str.get_dummies(',')
df = df.groupby('user_id').max()

## Extracting all of the users and the pois

In [6]:
listofusers = pd.DataFrame(checkin_data, columns= ['user_id']).groupby('user_id').max().sample(frac=1)
listofpois = pd.DataFrame(checkin_data, columns= ['poi_id', 'latitude', 'longitude']).groupby('poi_id').max().sample(frac=1)

userarray = listofusers.index.to_numpy()
poiarray = listofpois.index.to_numpy()

userdataframe = pd.DataFrame(userarray, columns = ['Users'])
poidataframe = pd.DataFrame(poiarray, columns = ['Poi'])

In [ ]:
#REDUNDANT, i think

rows_list = []
for i in range(len(poidataframe)):
    longitude = listofpois[listofpois.loc[i, "Poi"]]
    rows_list.append(dict1)

latlong = pd.DataFrame(rows_list) 
    

In [7]:
dot = userdataframe.merge(poidataframe, how='cross')

## Extracting ground_truth from incidence matrix

In [8]:
rows_list = []
for i in range(len(dot)):
    temp = df[dot.loc[i, "Poi"]][dot.loc[i, "Users"]]
    dict1 = {'ground_truth':float(temp)}
    rows_list.append(dict1)

groundtruth = pd.DataFrame(rows_list) 


In [9]:
result = pd.concat([dot, groundtruth], axis=1)

Define the model

In [99]:
class EmbModel(Model):
    def __init__(self):
        super(EmbModel, self).__init__()
        self.model = self.init_model()
        self.d_steps = 16
        
    def call(self, inputs):
        return
    
    def init_model(self):
        poi_latitude_input = keras.Input(shape=(1,), name='poi_latitude')
        poi_longitude_input = keras.Input(shape=(1,), name='poi_longitude')
        poi_concat_input = tf.keras.layers.Concatenate(axis=-1)([poi_latitude_input, poi_longitude_input])
        #input_length:  #This is the length of input sequences, as you would define for any input layer of a Keras model. 
                        #For example, if all of your input documents are comprised of 1000 words, this would be 1000
        #input_dim: 
                        #This is the size of the vocabulary in the text data. 
                        #For example, if your data is integer encoded to values between 0-10, then the size of the vocabulary would be 11 words.
        poi_emb = layers.Dense(50)(poi_concat_input)
        #poi_reshape = layers.Reshape((1,10))(poi_emb)
    
        user_input = keras.Input(shape=(1,), name='user_id')
        #user_dense = keras.layers.Dense(2)(user_input)
        user_emb = layers.Dense(50)(user_input)
        
                                    
        dot = layers.Dot(axes=(1))([poi_emb, user_emb])
        
        
        model = Model([[poi_latitude_input, poi_longitude_input], user_input], dot)
        model.summary()
        return model
    
    def compile_model(self, optimizer):
        super(EmbModel, self).compile(run_eagerly=True)
        self.optimizer = optimizer
        
    def train_step(self, data):
        if len(data) == 3:
            real_data, labels, sample_weight = data
        else:
            sample_weight = None
            real_data, labels = data
        user_data = real_data[0]
        latlong_data = real_data[1]
        
        for i in range(self.d_steps):
            
            with tf.GradientTape() as tape:
                
                #print(latlong_data[0])
                #print(latlong_data[1])
                #print(user_data)
                
                dotproduct = self.model([[latlong_data[0], latlong_data[1]], user_data])
                #print(dotproduct)
                # Loss function = ||S-GroundTruth|| 
                loss = tf.math.abs(tf.subtract(tf.cast(dotproduct, tf.float64), labels))
                #print(loss)
            d_gradient = tape.gradient(loss, self.model.trainable_variables)
            self.optimizer.apply_gradients(zip(d_gradient, self.model.trainable_variables))
        return {'loss': loss}


In [100]:
model = EmbModel()

Model: "functional_31"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
poi_latitude (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
poi_longitude (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
concatenate_19 (Concatenate)    (None, 2)            0           poi_latitude[0][0]               
                                                                 poi_longitude[0][0]              
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 1)]          0                                

In [12]:
print(listofpois)

                           latitude   longitude
poi_id                                         
4b763f1df964a5200e452ee3  35.510588  139.631317
4e176814b61c3f2c4879279a  13.726529  100.530999
50852ad2e4b08785ea69a703   35.67134  139.702852
4c73882bf3279c74a2dfb32d  35.439519  139.644728
4b36fbbef964a520d33e25e3  35.711149  139.774455
...                             ...         ...
4bb3649635f0c9b68049bc83  18.768972   98.974961
4d9c64e45c33a35dbccce8a0  51.518192   -0.116432
4cfebb7dd7206ea8b5d64f69  34.665246  135.529924
4b625594f964a520e4422ae3  42.341927   -71.05563
51e06112498e02ca8d08debf  19.421879  -99.116526

[2896 rows x 2 columns]


In [13]:
print([poidataframe.loc[0, "Poi"]])
print(listofpois.loc['4c576c0ccc96c9b6a70f792e']['latitude'])

['4b763f1df964a5200e452ee3']
37.843275


In [14]:
rows_list = []
for i in range(len(dot)):
    temp = dot.loc[i, "Poi"]
    latitude = listofpois.loc[temp]['latitude']
    longitude = listofpois.loc[temp]['longitude']
    dict1 = {'latitude':latitude, 'longitude':longitude}
    rows_list.append(dict1)
    #latitude = poiarray[i]
latlong = pd.DataFrame(rows_list)

## Creating dataset

In [15]:
userdot = pd.DataFrame(dot, columns= ['Users'])
latlong['latitude'] = pd.to_numeric(latlong['latitude'])
latlong['longitude'] = pd.to_numeric(latlong['longitude'])
dataset = pd.concat([userdot, latlong], axis=1)

In [16]:
dataset_numpy = dataset.to_numpy()
labels_numpy = groundtruth.to_numpy()

In [17]:
print(type(dataset_numpy[0][0]))
print(type(dataset_numpy[0][1]))
print(type(dataset_numpy[0][2]))
print(type(labels_numpy[0][0]))
#print(labels_numpy)

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [18]:
print(userdot)

       Users
0        451
1        451
2        451
3        451
4        451
...      ...
72395  29150
72396  29150
72397  29150
72398  29150
72399  29150

[72400 rows x 1 columns]


In [19]:
dataset1 = tf.convert_to_tensor(
    userdot, dtype=None, dtype_hint=None, name=None)
dataset2 = tf.convert_to_tensor(
    latlong, dtype=None, dtype_hint=None, name=None)
labels = tf.convert_to_tensor(
    labels_numpy, dtype=None, dtype_hint=None, name=None)

In [20]:
print(dataset2)

tf.Tensor(
[[ 35.510588 139.631317]
 [ 13.726529 100.530999]
 [ 35.67134  139.702852]
 ...
 [ 34.665246 135.529924]
 [ 42.341927 -71.05563 ]
 [ 19.421879 -99.116526]], shape=(72400, 2), dtype=float64)


In [101]:
import sys
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99, epsilon=1e-8)

model.compile(
    optimizer
)

#Train_data, [dataset.user_id, dataset.poi_id]. Label: ground_truth
model.fit([dataset1, dataset2], labels, epochs = 20, validation_split = 0.2, batch_size=2)

Epoch 1/20
28953/28960 [============================>.] - ETA: 0s - loss: 3956.0721WARNING:tensorflow:Layer emb_model_19 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

28960/28960 [==============================] - 52s 2ms/step - loss: 3954.9794 - val_loss: 0.0000e+00
Epoch 2/20
28960/28960 [==============================] - 54s 2ms/step - loss: 105.7293 - val_loss: 0